In [1]:
import json
from collections import defaultdict

from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, clear_output
import ipywidgets as widgets
import plotly.graph_objs as go
import plotly
import numpy as np
import numpy.linalg as nla

import sim_lib.util as util

init_notebook_mode(connected=True)

In [2]:
ntwk_data = {}
with open('ntwk_data/ntwk_data.json', 'r') as ntd:
    ntwk_data = json.loads(ntd.read())

In [3]:
class NtwkCompVis:
    def __init__(self, data):
        
        self.data = data

        r_vals = list(self.data.keys())
        p_vals = list(self.data[r_vals[0]].keys())
        
        # Welfare by p
        self.by_p_header = widgets.HTML(
                            value="<b>Average welfare by p</b>",
                            placeholder='Some HTML',
                            description='')
        self.by_p_r = widgets.SelectMultiple(
                                    options=r_vals,
                                    description='Resources per vertex',
                                    disabled=False)
        
        # Welfare by epoch
        self.by_epoch_header = widgets.HTML(
                            value="<b>Average welfare by epoch</b>",
                            placeholder='Some HTML',
                            description='')
        self.by_epoch_p = widgets.SelectMultiple(
                                    options=p_vals,
                                    description='Prob info trans',
                                    disabled=False)
        self.by_epoch_r = widgets.Select(
                                    options=r_vals,
                                    description='Resources per vertex',
                                    disabled=False)

        # Degree vs util transfer
        self.degree_tf_header = widgets.HTML(
                            value="<b>Utility transfered by degree</b>",
                            placeholder='Some HTML',
                            description='')
        self.degree_tf_p = widgets.Select(
                                    options=p_vals,
                                    description='Prob info trans',
                                    disabled=False)
        self.degree_tf_r = widgets.Select(
                                    options=r_vals,
                                    description='Resources per vertex',
                                    disabled=False)
        
        # Gain per interaction initiated
        self.int_gain_header = widgets.HTML(
                            value="<b>Gain / Interactions Initiated</b>",
                            placeholder='Some HTML',
                            description='')
        self.int_gain_r = widgets.SelectMultiple(
                                    options=r_vals,
                                    description='Resources per vertex',
                                    disabled=False)
        
        self.general_header = widgets.HTML(
                            value="<b>General settings</b>",
                            placeholder='Some HTML',
                            description='')
        self.ntwk_types = widgets.SelectMultiple(
                                    options=['rl', 'ws', 'er', 'cm', 'kg'],
                                    description='Network type',
                                    disabled=False)
        self.plot_selection = widgets.SelectMultiple(
                                    options=['by_p', 'by_epoch', 'gain', 'degree_tf'],
                                    description='Plots to render',
                                    disabled=False)

        # Update button
        self.plot_button = widgets.Button(
                                    description='Plot',
                                    disabled=False,
                                    button_style='',
                                    tooltip='Plots by selected parameters')
        self.plot_button.on_click(self.plot)
        
        self.visuals = [self.by_p_header, self.by_p_r,
                        self.by_epoch_header, self.by_epoch_p, self.by_epoch_r,
                        self.int_gain_header, self.int_gain_r,
                        self.degree_tf_header, self.degree_tf_p, self.degree_tf_r,
                        self.general_header, self.ntwk_types, self.plot_selection,
                        self.plot_button]
        
        self.render_visuals()
        
    def render_visuals(self):

        # Render widgets
        for vis in self.visuals:
            display(vis)

    def plot(self, button):
        
        # Reset widgets
        clear_output()
        self.render_visuals()
        
        by_p_r = self.by_p_r.value
        
        by_epoch_p = self.by_epoch_p.value
        by_epoch_r = self.by_epoch_r.value
        
        int_gain_r = self.int_gain_r.value
        
        degree_tf_p = self.degree_tf_p.value
        degree_tf_r = self.degree_tf_r.value
        
        ntwk_types = self.ntwk_types.value
        plots = self.plot_selection.value
        
        by_p = {}
        by_epoch = {}
        by_gain = {}
        degree_tf = {}
        
        for ntwk in ntwk_types:
            # plot by p
            if 'by_p' in plots:
                by_p_uts = self.plot_by_p(self.data, by_p_r, ntwk)
                by_p[ntwk] = by_p_uts
            
            # plot by epoch
            if 'by_epoch' in plots:
                by_epoch_uts = self.plot_by_epoch(self.data, by_epoch_r, by_epoch_p, ntwk)
                by_epoch[ntwk] = by_epoch_uts
            
            # plot by interactions initiated over actual path length
            if 'gain' in plots:
                gain_ratios = self.plot_int_gain_ratios(self.data, int_gain_r, ntwk)
                by_gain[ntwk] = gain_ratios
                
            if 'degree_tf' in plots:
                degree_transfers = self.plot_degree_tf(self.data, degree_tf_p, degree_tf_r, ntwk)
                degree_tf[ntwk] = degree_transfers
                
        #Create chart comparing all network types
        if 'by_p' in plots and len(ntwk_types) > 1:
            self.render_plot_by_p(by_p)
        if 'by_epoch' in plots and len(ntwk_types) > 1:
            self.render_plot_by_epoch(by_epoch, by_epoch_r)
        if 'gain' in plots and len(ntwk_types) > 1:
            self.render_plot_int_gain_ratios(by_gain)
        if 'degree_tf' in plots and len(ntwk_types) > 1:
            self.render_plot_degree_tf(degree_tf, degree_tf_p, degree_tf_r)
            
    def plot_by_p(self, data, by_p_r, ntwk):
        by_p_utils = []
        for r in by_p_r:
            r_utils = []
            for p, p_data in data[r].items():
                r_utils.append((p, [ sum(uts[-1]) / len(uts[-1])
                                    for uts in p_data['utils'][ntwk] ]))
            by_p_utils.append((r, r_utils))

        self.render_plot_by_p({ntwk : by_p_utils})
        return by_p_utils
        
    def render_plot_by_p(self, plot_data):        
        fig = go.Figure()

        colors = plotly.colors.DEFAULT_PLOTLY_COLORS
        
        comparison = (len(plot_data) > 1)
        
        i_idx = 0
        for ntwk, utils in plot_data.items():
            for j_idx, r_data in enumerate(utils):
                color = colors[(i_idx + j_idx) % len(colors)]

                r_val, r_utils = r_data

                #Add averages trace
                p_vals = [ p for (p, uts) in r_utils ]
                ut_vals = [ np.average(uts) for (p, uts) in r_utils ]

                #Error bars
                std_vals = [ np.std(uts) for (p, uts) in r_utils ]

                ntwk_name = f' ntwk={ntwk}' if comparison else ''
                fig.add_trace(go.Scatter(x=p_vals,
                    y=ut_vals, mode='lines+markers', hoverinfo='skip',
                    name=f'avg social welfare rate (r={r_val}{ntwk_name})', marker=dict(color=color),
                    error_y=dict(type='data', array=std_vals, visible=True)))

                #Add individual points trace
                indiv_p_vals = []
                indiv_ut_vals = []
                for (p, uts) in r_utils:
                    indiv_ut_vals.extend(uts)
                    indiv_p_vals.extend([p] * len(uts))
                fig.add_trace(go.Scatter(x=indiv_p_vals,
                    y=indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.3,
                    marker=dict(color=color), name=f'social welfare per iter (r={r_val}{ntwk_name})'))
            i_idx += 1

        fig.layout.update(showlegend=False)
        ntwk_name = 'Comparison'
        if not comparison:
            ntwk_name = list(plot_data.keys())[0]
        plot_title = f'Average Social Welfare vs p ({ntwk_name})'
        fig.layout.update(title=plot_title, showlegend=True,
            xaxis=dict(title='p'), yaxis=dict(title='avg social welfare'),
            plot_bgcolor='rgba(0,0,0,0)')

        iplot(fig)
        
    def plot_by_epoch(self, data, by_epoch_r, by_epoch_p, ntwk):
        by_epoch_utils = []
        epoch_r_data = data[by_epoch_r]
        for p in by_epoch_p:
            p_utils = []
            epoch_p_utils = epoch_r_data[p]['utils'][ntwk]
            for it_uts in epoch_p_utils:
                p_utils.append([(ep, sum(ep_uts) / len(ep_uts)) for ep, ep_uts in enumerate(it_uts)])
            by_epoch_utils.append((p, p_utils))

        self.render_plot_by_epoch({ntwk : by_epoch_utils}, by_epoch_r)
        return by_epoch_utils
            
    def render_plot_by_epoch(self, plot_data, r_val):
        fig = go.Figure()

        colors = plotly.colors.DEFAULT_PLOTLY_COLORS
        
        comparison = (len(plot_data) > 1)
        
        max_epoch_count = -1
        
        i_idx = 0
        for ntwk, utils in plot_data.items():
            for j_idx, p_data in enumerate(utils):
                color = colors[(i_idx + j_idx)  % len(colors)]

                p_val, p_utils = p_data

                #Add averages trace
                ep_uts = defaultdict(lambda : [])
                for ut_per_ep in p_utils:
                    for (ep, ut) in ut_per_ep:
                        ep_uts[ep].append(ut)

                ep_vals = sorted(list(ep_uts.keys()))
                ut_vals = [ np.average(ep_uts[ep]) for ep in ep_vals ]
                std_vals = [ np.std(ep_uts[ep]) for ep in ep_vals ]
                
                ntwk_name = f' ntwk={ntwk}' if comparison else ''
                fig.add_trace(go.Scatter(x=ep_vals,
                    y=ut_vals, mode='lines+markers', hoverinfo='skip',
                    name=f'avg social welfare rate (p={p_val}{ntwk_name})', marker=dict(color=color),
                    error_y=dict(type='data', array=std_vals, visible=True)))

                #Add individual points trace
                indiv_ep_vals = []
                indiv_ut_vals = []
                for ep in ep_vals:
                    uts = ep_uts[ep]
                    indiv_ut_vals.extend(uts)
                    indiv_ep_vals.extend([ep] * len(uts))
                fig.add_trace(go.Scatter(x=indiv_ep_vals,
                    y=indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.3,
                    marker=dict(color=color), name=f'social welfare per iter (p={p_val}{ntwk_name})'))
                
                max_epoch_count = max(max_epoch_count, len(ep_vals))
            i_idx += 1

        fig.layout.update(showlegend=False)
        ntwk_name = 'Comparison'
        if not comparison:
            ntwk_name = list(plot_data.keys())[0]
        plot_title = f"Average Social Welfare vs Epoch # (r={r_val} {ntwk_name})"
        fig.layout.update(title=plot_title, showlegend=True,
            xaxis=dict(title='Epoch #'), yaxis=dict(title='avg social welfare'),
            plot_bgcolor='rgba(0,0,0,0)')
        fig.update_xaxes(range=[0, max_epoch_count])

        iplot(fig)
        
    def plot_int_gain_ratios(self, data, int_gain_r, ntwk):
        int_gain_ratios = []
        for r in int_gain_r:
            r_ratios = []
            for p, p_data in data[r].items():
                p_ratios = []
                ser_graphs = data[r][p]['graphs'][ntwk] 
                for idx, fsg in enumerate(ser_graphs):
                    isg = data[r][p]['init_graphs'][ntwk][idx]
                    init_g = util.json_to_graph(isg)
                    final_g = util.json_to_graph(fsg)
                    init_uts = { v.vnum : v.utility for v in init_g.vertices }
                    final_uts = { v.vnum : v.utility for v in final_g.vertices }
                    init_prov = { v.vnum : v.provider for v in init_g.vertices }
                    final_prov = { v.vnum : v.provider for v in final_g.vertices }
                    ratios = []
                    for v in final_g.vertices:
                        gain = final_uts[v.vnum] - init_uts[v.vnum]
                        if v.init_ints == 0 or gain == 0:
                            continue
                        ratios.append(gain / v.init_ints)
                    p_ratios.append(np.average(ratios))
                r_ratios.append((p, p_ratios))
            int_gain_ratios.append((r, r_ratios))

        self.render_plot_int_gain_ratios({ntwk : int_gain_ratios})
        return int_gain_ratios
        
    def render_plot_int_gain_ratios(self, plot_data):
        fig = go.Figure()

        colors = plotly.colors.DEFAULT_PLOTLY_COLORS
        
        comparison = (len(plot_data) > 1)
        
        i_idx = 0
        for ntwk, ratios in plot_data.items():
            for j_idx, r_data in enumerate(ratios):
                color = colors[(i_idx + j_idx) % len(colors)]

                r_val, r_ratios = r_data

                #Add averages trace
                p_vals = [ p for (p, ratios) in r_ratios ]
                ratio_vals = [ np.average(ratios) for (p, ratios) in r_ratios ]

                #Error bars
                std_vals = [ np.std(ratios) for (p, ratios) in r_ratios ]

                ntwk_name = f' ntwk={ntwk}' if comparison else ''
                fig.add_trace(go.Scatter(x=p_vals,
                    y=ratio_vals, mode='lines+markers', hoverinfo='skip',
                    name=f'Gain / Interactions initiated (r={r_val}{ntwk_name})', marker=dict(color=color),
                    error_y=dict(type='data', array=std_vals, visible=True)))

                #Add individual points trace
                indiv_p_vals = []
                indiv_ratio_vals = []
                for (p, ratios) in r_ratios:
                    indiv_ratio_vals.extend(ratios)
                    indiv_p_vals.extend([p] * len(ratios))
                fig.add_trace(go.Scatter(x=indiv_p_vals,
                    y=indiv_ratio_vals, mode='markers', hoverinfo='skip', opacity=0.3,
                    marker=dict(color=color), name=f'ratio per iter (r={r_val}{ntwk_name})'))
            i_idx += 1

        fig.layout.update(showlegend=False)
        ntwk_name = 'Comparison'
        if not comparison:
            ntwk_name = list(plot_data.keys())[0]
        plot_title = f"Gain / Interactions initiated vs p ({ntwk_name})"
        fig.layout.update(title=plot_title, showlegend=True,
            xaxis=dict(title='p'), yaxis=dict(title='gain / interactions initiated'),
            plot_bgcolor='rgba(0,0,0,0)')

        iplot(fig)
        
    def plot_degree_tf(self, data, degree_tf_p, degree_tf_r, ntwk):
        in_transfers = [] # gains
        out_transfers = []
        sim_data = data[degree_tf_r][degree_tf_p]
        
        for idx, fsg in enumerate(sim_data['graphs'][ntwk]):
            
            # Add dicts for mapping degree : transfer lists
            in_transfers.append(defaultdict(lambda : []))
            out_transfers.append(defaultdict(lambda : []))
            
            isg = sim_data['init_graphs'][ntwk][idx]
            init_g = util.json_to_graph(isg)
            final_g = util.json_to_graph(fsg)
            init_uts = { v.vnum : v.utility for v in init_g.vertices }
            final_uts = { v.vnum : v.utility for v in final_g.vertices }
            init_prov = { v.vnum : v.provider for v in init_g.vertices }
            final_prov = { v.vnum : v.provider for v in final_g.vertices }
            for v in final_g.vertices:
                gain = final_uts[v.vnum] - init_uts[v.vnum]
                if v.init_ints == 0 or gain == 0:
                    continue
                in_transfers[idx][v.degree].append(gain)
                
            # Neighborhoods don't change now so construct static from init graph
            nbor_map = { v.vnum : [ nbor.vnum for nbor in v.nbors ] for v in init_g.vertices }
            vnum_map = { v.vnum : v for v in init_g.vertices }
            
            iter_out_tfs = defaultdict(lambda : 0)
            ut_map = sim_data['maps'][ntwk][idx]
            
            for vtx in ut_map: # Uses vnum to represent vtx
                nbors = nbor_map[int(vtx)]
                vtx_out_tfs = 0
                for nbor in nbors:
                    nbor = str(nbor)
                    if len(ut_map[nbor]['from']) < 2: # no gains in util
                        continue
                        
                    if ut_map[nbor]['from'][-2] != int(vtx):
                        continue
                    nbor_cur_ut = ut_map[nbor]['ut']
                    nbor_prev_ut = init_uts[int(nbor)]
                    vtx_out_tfs += (nbor_cur_ut - nbor_prev_ut)
                iter_out_tfs[vtx] += vtx_out_tfs
                
            for vtx, tfs in iter_out_tfs.items():
                out_transfers[idx][vnum_map[int(vtx)].degree].append(tfs)
                
        for itf_dict in in_transfers:
            for deg, tfs in itf_dict.items():
                itf_dict[deg] = np.average(tfs)
        for otf_dict in out_transfers:
            for deg, tfs in otf_dict.items():
                otf_dict[deg] = np.average(tfs)

        self.render_plot_degree_tf({ntwk : (in_transfers, out_transfers)}, degree_tf_p, degree_tf_r)
        return (in_transfers, out_transfers)
    
    def render_plot_degree_tf(self, plot_data, p_val, r_val):
        fig = go.Figure()

        colors = plotly.colors.DEFAULT_PLOTLY_COLORS
        
        comparison = (len(plot_data) > 1)
        
        i_idx = 0
        for ntwk, transfers in plot_data.items():
            in_color = colors[(i_idx) % len(colors)]
            out_color = colors[(i_idx + 1) % len(colors)]

            in_tfs, out_tfs = transfers

            for direction, color, tfs in [('In', in_color, in_tfs), ('Out', out_color, out_tfs)]:
                tf_degs = np.array([ list(tf.keys()) for tf in tfs ])
                tf_degs = np.unique(np.hstack(tf_degs))

                tf_full = defaultdict(lambda : []) # Map of all degrees to all transfers
                for tf_dict in tfs:
                    for deg, tf_val in tf_dict.items():
                        tf_full[deg].append(tf_val)

                tf_avgs = [ np.average(tf_full[deg]) for deg in tf_degs ]
                #Error bars
                tf_std_vals = [ np.std(tf_vals) for (deg, tf_vals) in tf_full.items() ]

                ntwk_name = f' ntwk={ntwk}' if comparison else ''
                fig.add_trace(go.Scatter(x=tf_degs,
                    y=tf_avgs, mode='lines+markers', hoverinfo='skip',
                    name=f'{direction} transfers (r={r_val}{ntwk_name})', marker=dict(color=color),
                    error_y=dict(type='data', array=tf_std_vals, visible=True)))

                #Add individual points trace
                indiv_tf_degs = []
                indiv_tf_vals = []
                for (deg, tf_vals) in tf_full.items():
                    indiv_tf_vals.extend(tf_vals)
                    indiv_tf_degs.extend([deg] * len(tf_vals))
                fig.add_trace(go.Scatter(x=indiv_tf_degs,
                    y=indiv_tf_vals, mode='markers', hoverinfo='skip', opacity=0.3,
                    marker=dict(color=color), name=f'{direction} transfers per iter (r={r_val}{ntwk_name})'))
            i_idx += 2

        fig.layout.update(showlegend=False)
        ntwk_name = 'Comparison'
        if not comparison:
            ntwk_name = list(plot_data.keys())[0]
        plot_title = f"Degree vs Utility transfers ({ntwk_name})"
        fig.layout.update(title=plot_title, showlegend=True,
            xaxis=dict(title='degree'), yaxis=dict(title='utility transfered'),
            plot_bgcolor='rgba(0,0,0,0)')

        iplot(fig)

In [4]:
vis = NtwkCompVis(ntwk_data)

HTML(value='<b>Average welfare by p</b>', placeholder='Some HTML')

SelectMultiple(description='Resources per vertex', options=('128', '64', '32', '16', '8', '4', '2'), value=())

HTML(value='<b>Average welfare by epoch</b>', placeholder='Some HTML')

SelectMultiple(description='Prob info trans', index=(8,), options=('1.0', '0.9', '0.8', '0.7', '0.6', '0.5', '…

Select(description='Resources per vertex', index=4, options=('128', '64', '32', '16', '8', '4', '2'), value='8…

...

Select(description='Resources per vertex', options=('128', '64', '32', '16', '8', '4', '2'), value='128')

HTML(value='<b>General settings</b>', placeholder='Some HTML')

SelectMultiple(description='Network type', index=(0, 1, 2, 3, 4), options=('rl', 'ws', 'er', 'cm', 'kg'), valu…

SelectMultiple(description='Plots to render', index=(1,), options=('by_p', 'by_epoch', 'gain', 'degree_tf'), v…

Button(description='Plot', style=ButtonStyle(), tooltip='Plots by selected parameters')